# Min Cost Flow ILP

### Author: [Ben Rosenberg](https://benrosenberg.info)


### Imports

We begin by importing some relevant libraries. We import `ortools` to formulate and solve the ILP, and we import `time` to time the entire process of supplying the constraints solving the ILP.

In [1]:
from ortools.linear_solver import pywraplp as OR
import time

### Input data

Next, we define our input data. Recall that in the Min Cost Flow problem we have as an input a graph $G = (N, E)$, where $N$ is the set of nodes, each of which has supply $s(i)$ for all $i\in N$, and $E$ is the set of edges, each of which has a cost $c(i,j)$ and capacity $u(i,j)$ for all $(i,j)\in E$.

The input data below corresponds to a small example graph, seen below, in which the numbers inside the nodes are their supply/demand values, and the first element of the tuple corresponding to each edge denotes the cost and the second denotes the capacity:

![](mcf_graph.png)

In [2]:
nodes = range(4)

edges = [(0,1), (0,3), (2,1), (2,3)]

# cost[i,j] is the cost of sending one unit of flow along edge (i,j)
cost = {
    (0,1) : 0,
    (0,3) : 4,
    (2,1) : 2,
    (2,3) : 8
} 

# capacity[i,j] is the capacity of edge (i,j)
capacity = {
    (0,1) : 5,
    (0,3) : 2,
    (2,1) : 3,
    (2,3) : 2
}

# supply[i] is the supply (if positive) or demand (if negative) of node i
supply = [5, -6, 3, -2]

### Model Definition

Omitted.

In [3]:
start_time = time.time()

m = OR.Solver('MCF', OR.Solver.GLOP_LINEAR_PROGRAMMING)

# add variable f
f = {}
for (i,j) in edges:
    f[i,j] = m.NumVar(0, m.infinity(), 'f[{},{}]'.format(
        i,j
    ))

# add constraint representing supply/demand
for i in nodes:
    m.Add(sum(f[x,j] for (x,j) in edges if x == i) - 
          sum(f[j,x] for (j,x) in edges if x == i)
          == supply[i])

# add constraint on edge flows w.r.t. capacities
for (i,j) in edges:
    m.Add(f[i,j] <= capacity[i,j])

# add constraint on edge flows w.r.t. 0
for (i,j) in edges:
    m.Add(f[i,j] >= 0)
    
# set objective
m.Minimize(sum(cost[i,j] * f[i,j] for (i,j) in edges))

m.Solve()

end_time = time.time()

diff = time.gmtime(end_time - start_time)
print('\n[Total time used: {} minutes, {} seconds]'.format(diff.tm_min, diff.tm_sec))

print('Objective:', m.Objective().Value())


[Total time used: 0 minutes, 0 seconds]
Objective: 14.0


So we have our optimal objective. The optimal solution associated with that, in terms of flows on edges, is given below:

In [4]:
# optimal solution
for (i,j) in edges:
    print('f[{},{}] = {}'.format(
        i, j, f[i,j].solution_value()
    ))

f[0,1] = 3.0
f[0,3] = 2.0
f[2,1] = 3.0
f[2,3] = 0.0
